# Openpyxl demo using candidates' data for the 2023 federal elections

With Openpyxl, you can create Excel files, add sheets, add content and format the content as an Excel table with sortable columns.


In [1]:

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles import Font, Alignment, Border, Side



In [2]:

# get the list of election candidates data from the internet
import requests

url = "https://ogd-static.voteinfo-app.ch/v4/ogd/sd-t-17.02-NRW2023-kandidierende.json"

# Make a GET request to the opendata.swiss url
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    json_data = response.json()
    

## opendata.swiss

open.swiss is a publicly available web site with thousands of datasets in various formats (.csv, .xls, .json). In this demo file, we are using a json file. 

Here is a preview of the json data:

```json
"level_kantone":
[
{
"kanton_nummer":1,
"kanton_bezeichnung":"Zürich",
"liste_nummer_bfs":3,
"liste_nummer_kanton":"3",
"liste_bezeichnung":"GRÜNE",
"kandidat_nummer":10,
"kandidat_listenplatz_1":10,
"kandidat_listenplatz_2":null,
"name":"Bühlmann",
"vorname":"Claudia",
"geschlecht":"F",
"geburtsdatum":"06.01.1972",
"geburtsjahr":1972,
"wohnort_gemeinde_nummer":null,
"wohnort":"Wädenswil",
"beruf":"Stadträtin, Bio-Landwirtin, Arbeitsagogin",
"kandidat_status_id":null,
"kandidat_partei_id":13,
"stimmen_kandidat":null,
"flag_gewaehlt":null
},
...
```

In [3]:
import json
import pandas as pd


# Extract candidate data
candidate_data = json_data.get("level_kantone", [])

# Create a list to hold the extracted data
candidate_records = []

# Iterate through candidate data and flatten it into records
for candidate in candidate_data:
    candidate_record = {
        "liste_nummer_bfs": candidate.get("liste_nummer_bfs", None),
        "liste_bezeichnung": candidate.get("liste_bezeichnung", None),
        "kandidat_nummer": candidate.get("kandidat_nummer", None),
        "name": candidate.get("name", None),
        "vorname": candidate.get("vorname", None),
        "geschlecht": candidate.get("geschlecht", None),
        "geburtsdatum": candidate.get("geburtsdatum", None),
        "wohnort": candidate.get("wohnort", None),
        "beruf": candidate.get("beruf", None),
        "kanton_nummer": candidate.get("kanton_nummer", None),
        "kanton_bezeichnung": candidate.get("kanton_bezeichnung", None),
    }
    candidate_records.append(candidate_record)

# Create a DataFrame from the extracted data
df = pd.DataFrame(candidate_records)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5928 entries, 0 to 5927
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   liste_nummer_bfs    5928 non-null   int64 
 1   liste_bezeichnung   5928 non-null   object
 2   kandidat_nummer     5928 non-null   int64 
 3   name                5928 non-null   object
 4   vorname             5928 non-null   object
 5   geschlecht          5924 non-null   object
 6   geburtsdatum        5924 non-null   object
 7   wohnort             5924 non-null   object
 8   beruf               5924 non-null   object
 9   kanton_nummer       5928 non-null   int64 
 10  kanton_bezeichnung  5928 non-null   object
dtypes: int64(3), object(8)
memory usage: 509.6+ KB


In [4]:
# Create a new workbook
wb = Workbook()

# Get the active sheet
ws = wb.active

# Set the title of the sheet
ws.title = "Kandidierende"

# Create a list of column names

columns = [

    "liste_nummer_bfs",
    "liste_bezeichnung",
    "kandidat_nummer",
    "name",
    "vorname",
    "geschlecht",
    "geburtsdatum",
    "wohnort",
    "beruf",
    "kanton_nummer",
    "kanton_bezeichnung",
]

# Iterate through the columns and set the values of the first row
for col in range(len(columns)):
    ws.cell(row=1, column=col + 1).value = columns[col]

# Iterate through the DataFrame and set the values of the rows
for row in range(len(df)):
    for col in range(len(columns)):
        ws.cell(row=row + 2, column=col + 1).value = df.iloc[row, col]

# Create an Excel table from the data in the worksheet
table = Table(displayName="data", ref=ws.dimensions)

# Create a style for the table
style = TableStyleInfo(
    name="TableStyleMedium9", 
    showRowStripes=True
)

# Add the style to the table
table.tableStyleInfo = style

# Add the table to the worksheet
ws.add_table(table)

In [5]:
# Save the workbook
wb.save("../data/2023_candidates.xlsx")

### Percentages of women in each party for the 2023 Swiss election (lists from Zürich)

In [6]:
# Calculate the percentage of females in each list
pivot_df = df.pivot_table(index=['kanton_bezeichnung','liste_bezeichnung'], columns='geschlecht', aggfunc='size', fill_value=0)
pivot_df['Total'] = pivot_df['F'] + pivot_df['M']
pivot_df['Percentage Female'] = (pivot_df['F'] / pivot_df['Total']) * 100


In [7]:

pd.options.display.float_format = '{:.1f}%'.format
pivot_df.loc["Zürich",:].sort_index()



geschlecht,F,M,Total,Percentage Female
liste_bezeichnung,,,,
AL – Alternative Liste,20,16,36,55.6%
Aufrecht Zürich,15,21,36,41.7%
Die Mitte,19,17,36,52.8%
Die Mitte – Die Erfahrenen,7,29,36,19.4%
Die Mitte – Die Junge Mitte Nord-Ost,13,23,36,36.1%
Die Mitte – Die Junge Mitte Süd-West,12,24,36,33.3%
Die Mitte – Frauen,36,0,36,100.0%
Die Mitte – Wirtschaft und Gesellschaft AWG,8,28,36,22.2%
Digital-liberale Allianz – Piratenpartei Schweiz,5,13,18,27.8%
